In [1]:
import pandas as pd
import pyarrow
import numpy as np 


In [2]:
parquet_test_identity = '../datasets/test_identity.parquet'
parquet_test_trans = '../datasets/test_transaction.parquet'


In [3]:
df_trans = pd.read_parquet(parquet_test_trans)
df_ident = pd.read_parquet(parquet_test_identity)
df_merge = df_trans.merge(df_ident, on="TransactionID", how="left")
df_merge.head()

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,DeviceInfo
0,3663549,18403224,31.95,W,10409,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3663550,18403263,49.00,W,4272,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3663551,18403310,171.00,W,4476,574.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3663552,18403310,284.95,W,10989,360.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3663553,18403317,67.95,W,18018,452.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_merge["hour"] = (df_merge["TransactionDT"] // 3600) % 24
df_merge['amt_log'] = np.log1p(df_merge['TransactionAmt'])
df_merge['card_txn_count'] = df_merge.groupby('card1')['TransactionID'].transform('count')
cols = ['TransactionID','TransactionAmt','card1','C1','C2','C13','V12','hour','amt_log','card_txn_count']
df_merge[df_merge['TransactionID'] == 3663552][cols]

,TransactionID,TransactionAmt,card1,C1,C2,C13,V12,hour,amt_log,card_txn_count
3,3663552,284.95,10989,5.0,2.0,7.0,1.0,0,5.655817,839


In [16]:
import requests, json

url = "http://192.168.31.16:9696"
# single-record prediction
{
    "data":[
    {
      "TransactionID": 3663552,
      "TransactionAmt": 284.95,
      "card1": 10989,
      "C1": 5,
      "C2": 2.0,
      "C13": 7,
      "V12": 1,
      "hour": 0,
      "amt_log": 5.655817,
      "card_txn_count": 839
    }
  ]
}
response = requests.post(url, json=data)
print(response.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)